# Library

In [1]:
#import sys
import pandas as pd
import numpy as np
import random
import pyspark
import itertools
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
import pickle
import statsmodels.api as sm

from pyspark import SparkContext, SQLContext

from math import sqrt
from time import time as ttt

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier as DTC_spark
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import joblib
from joblib import parallel_backend
from joblib import Parallel, delayed
from joblib import parallel_backend


# Upload

In [2]:
spark = SparkSession.builder.master("local").appName("spark_app_1234").getOrCreate()
sc = spark.sparkContext
spark
d0 = (
    spark
    .read
    .format("csv") 
    .option("header","true") 
    .option("inferSchema","true") 
    .load("gs://mas-a5-storage-1/notebooks/jupyter/application_train.csv")
)

d1 = d0.filter(d0.DAYS_EMPLOYED != 365243).select('TARGET','DAYS_EMPLOYED')

print(d1.rdd.getNumPartitions())

4


# Function

In [3]:
def prepare_spark_data(n_part, k_mult):
    '''
    takes 'DAYS_EMPLOYED and 'TARGET'  from d1 (alreay filtered)
    rearrane in n partitions (if n==0 keeps initial number of partitions)
    prints final shape/ partition
    returns d2 - spark df
    '''
    data = d1
    data_new = data # first step in the cycle
    for i in range(k_mult-1):
        data_tmp = data.select('TARGET', \
                     f.col('DAYS_EMPLOYED')*(f.lit(0.9995) + f.rand()/1000)).\
                   toDF('TARGET','DAYS_EMPLOYED')
        data_tmp = data_tmp.select('TARGET', f.floor('DAYS_EMPLOYED'))
        data_new = data_new.union(data_tmp)

    assembler = VectorAssembler(inputCols=["DAYS_EMPLOYED"], 
                        outputCol="DAYS_EMPLOYED_vect")
    d2 = assembler.transform(data_new)
    if n_part != 0:
        d2 = d2.repartition(n_part)       
    #print(f'n-partitions initial: {d2.rdd.getNumPartitions()}; df size: {d2.count()}\n')

    return d2

# Params an run

In [ ]:
## !! mind the fn NOT TO rewrite results
fn = '4cpu_by_8n__4m'
size_mult = [1, 2, 5, 10, 15, 20, 30, 50, 100]
partitions = [0, 2, 4, 6, 8, 16, 24] 
# initial, 2**i incl n_nodes till n_cpu*n_nodes and n_cpu*n_nodes
n_iter = 3

print('d1-size', d1.count())
n_part_base = d1.rdd.getNumPartitions()
print('n-partitions initial', n_part_base, '\n')

rd1 = {}
for k_size_mult in size_mult:
    rd2 = {}
    for n_part in partitions:
        df = prepare_spark_data(n_part, k_size_mult)
        df.cache()
        print('======================================================================')
        print(f'=== size_mult={k_size_mult}; \
        n_part_req={n_part}; n_part_act={df.rdd.getNumPartitions()}; df_count={df.count()} ===\n')
        times = [0 for i in range(n_iter)]
        models = {}
        for i in range(n_iter):
            dt = DTC_spark(labelCol="TARGET",
                       featuresCol="DAYS_EMPLOYED_vect",
                       minInfoGain=0.0001,
                       impurity='entropy',
                       maxDepth=14, maxBins=2**14, # it differs from scikit learn - it means number of canidate split points
                       #minInstancesPerNode = 1,
                       #checkpointInterval = 10
                       )
           
            t0 = ttt()
            model = dt.fit(df)
            t1 = ttt()
            times[i] = t1-t0
            models[i] = model 
            print('model', i, 'build time', round(times[i],2), '\n', model)
        rd2[n_part] = (models, times)
    rd1[k_size_mult] = rd2

result = rd1  

d1-size 252137
n-partitions initial 4 



=== size_mult=1;         n_part_req=0; n_part_act=2; df_count=252137 ===



model 0 build time 10.51 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_3c772b79f38d, depth=14, numNodes=123, numClasses=2, numFeatures=1


model 1 build time 7.34 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_2383f1a0e8b4, depth=14, numNodes=123, numClasses=2, numFeatures=1


model 2 build time 6.88 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_738bc0779bd7, depth=14, numNodes=123, numClasses=2, numFeatures=1


...

model 2 build time 20.46 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_96987f4ea181, depth=9, numNodes=31, numClasses=2, numFeatures=1


=== size_mult=100;         n_part_req=24; n_part_act=24; df_count=25213700 ===



model 0 build time 24.16 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_5760a8b20b79, depth=11, numNodes=41, numClasses=2, numFeatures=1


model 1 build time 24.86 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_39f92fdb9d1d, depth=11, numNodes=41, numClasses=2, numFeatures=1


model 2 build time 25.24 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_d110b972cb23, depth=11, numNodes=41, numClasses=2, numFeatures=1


# Results

In [5]:
di = result
df_res_time = pd.DataFrame()
df_res_nodes = pd.DataFrame()
for p1 in di.keys():
    for p2 in di[p1].keys():
        df_res_time.loc[p1, p2] = np.round(np.mean(di[p1][p2][1]), 1)
        m_tmp = di[p1][p2][0]
        n_nodes = []
        for i,_ in enumerate(m_tmp):
            tmp = f'{m_tmp[i]}'.split(' ')
            print(i, tmp)
            n_nodes.append([int(x.split('=')[1][:-1]) for x in tmp if x[:4]== 'numN'][0])
        df_res_nodes.loc[p1, p2] = np.round(np.mean(n_nodes), 1)

df_res_time.to_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_t.csv')
df_res_nodes.to_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_n.csv')

display(df_res_time)      
display(df_res_nodes)     

0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_3c772b79f38d,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
1 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_2383f1a0e8b4,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
2 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_738bc0779bd7,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_bfafa45d2530,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
1 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_3cc53f0e6076,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
2 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_81648d65bb23,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_4137bf38d637,', 'depth=14,', 'numNodes=123,', 'numC

,0,2,4,6,8,16,24
1,8.2,6.7,7.9,9.1,10.2,15.6,20.8
2,14.3,8.9,9.8,5.4,6.4,17.4,27.8
5,45.5,13.8,10.5,8.0,6.6,5.4,6.3
10,48.4,20.0,12.3,8.8,8.9,6.6,7.6
15,61.2,24.5,14.6,11.8,9.8,7.8,8.6
20,84.0,29.3,17.2,13.5,11.8,7.7,8.1
30,135.2,39.0,22.2,16.3,14.0,9.5,11.4
50,226.8,49.5,29.2,21.3,18.0,12.7,14.0
100,463.1,91.3,56.2,39.5,33.5,21.7,24.8


,0,2,4,6,8,16,24
1,123.0,123.0,123.0,123.0,123.0,123.0,123.0
2,149.0,159.0,143.0,97.0,121.0,127.0,131.0
5,91.0,97.0,123.0,121.0,93.0,95.0,75.0
10,85.0,93.0,69.0,71.0,81.0,61.0,77.0
15,63.0,43.0,39.0,65.0,43.0,53.0,47.0
20,63.0,61.0,59.0,47.0,63.0,65.0,37.0
30,57.0,31.0,37.0,39.0,45.0,45.0,53.0
50,31.0,43.0,35.0,65.0,37.0,55.0,37.0
100,35.0,39.0,39.0,53.0,37.0,31.0,41.0


In [6]:
fn = '4cpu_by_8n__4m'

tmp = pd.read_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_t.csv')
tmp

,Unnamed: 0,0,2,4,6,8,16,24
0,1,8.2,6.7,7.9,9.1,10.2,15.6,20.8
1,2,14.3,8.9,9.8,5.4,6.4,17.4,27.8
2,5,45.5,13.8,10.5,8.0,6.6,5.4,6.3
3,10,48.4,20.0,12.3,8.8,8.9,6.6,7.6
4,15,61.2,24.5,14.6,11.8,9.8,7.8,8.6
5,20,84.0,29.3,17.2,13.5,11.8,7.7,8.1
6,30,135.2,39.0,22.2,16.3,14.0,9.5,11.4
7,50,226.8,49.5,29.2,21.3,18.0,12.7,14.0
8,100,463.1,91.3,56.2,39.5,33.5,21.7,24.8


In [7]:
fn = '4cpu_by_8n__4m'
tmp = pd.read_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_n.csv')
tmp

,Unnamed: 0,0,2,4,6,8,16,24
0,1,123.0,123.0,123.0,123.0,123.0,123.0,123.0
1,2,149.0,159.0,143.0,97.0,121.0,127.0,131.0
2,5,91.0,97.0,123.0,121.0,93.0,95.0,75.0
3,10,85.0,93.0,69.0,71.0,81.0,61.0,77.0
4,15,63.0,43.0,39.0,65.0,43.0,53.0,47.0
5,20,63.0,61.0,59.0,47.0,63.0,65.0,37.0
6,30,57.0,31.0,37.0,39.0,45.0,45.0,53.0
7,50,31.0,43.0,35.0,65.0,37.0,55.0,37.0
8,100,35.0,39.0,39.0,53.0,37.0,31.0,41.0
